In [1]:
import numpy as np
import pptk

In [11]:
%matplotlib auto

Using matplotlib backend: MacOSX


In [51]:
pip install pptk

ERROR: Could not find a version that satisfies the requirement pptk (from versions: none)
ERROR: No matching distribution found for pptk
Note: you may need to restart the kernel to use updated packages.


In [36]:
file_data_path='/Users/carotenuto/Master Radboud/MscProj/preds_pcl_all_c256_l10/preds/vis/pred_pt_00020.txt'
point_cloud= np.loadtxt(file_data_path,delimiter=',')
#mean_Z=np.mean(point_cloud,axis=0)[2]
#spatial_query=point_cloud[abs( point_cloud[:,2]-mean_Z)<1]
xyz=spatial_query[:,:3]
rgb=spatial_query[:,3:]

In [41]:
ax = plt.axes(projection='3d')

In [42]:
ax.scatter(xyz[:,0], xyz[:,1], xyz[:,2], c = rgb*255, s=1)
plt.show()

In [33]:
plt.show()

In [34]:
point_cloud.shape

(25892, 6)

In [40]:
point_cloud[0]

array([ 30.798434, -43.637868,  17.38005 ,   0.      ,   0.      ,
         0.      ])

In [124]:
file_data_path='/Users/carotenuto/Master Radboud/MscProj/preds_pcl_all_c256_l10/preds/vis/pred_pt_01510.txt'
pcl= np.loadtxt(file_data_path,delimiter=',')
col1 = pcl[:,3:]
file_data_path2='/Users/carotenuto/Master Radboud/MscProj/preds_pcl_all_c256_l10/gt_vis/gt_pt_01510.txt'
pcl2= np.loadtxt(file_data_path2,delimiter=',')
col2 = pcl2[:,3:]

In [125]:
col = np.maximum(col1, col2)

In [126]:
for i in range(len(col)):
    if col[i,0] == 1 and col[i,1] == 1:
        col[i,0] = 0.0

In [127]:
v = pptk.viewer(pcl[:,:3], col, show_axis=False)
v.set(point_size=0.5, show_axis=False, show_info=True)

(25892, 3)

In [112]:
file_data_path='/Users/carotenuto/Master Radboud/MscProj/preds_pcl_all_c256_l10/preds/vis/pred_01511.txt'
pcl= np.loadtxt(file_data_path,delimiter=',')
col1 = pcl[:,3:]
file_data_path2='/Users/carotenuto/Master Radboud/MscProj/preds_pcl_all_c256_l10/gt_vis/gt_01511.txt'
pcl2= np.loadtxt(file_data_path2,delimiter=',')
col2 = pcl2[:,3:]

In [115]:
col = np.zeros((col1.shape[0], col1.shape[1]))
for i in range(len(col)):
    col[i,2] = col1[i,1]

v = pptk.viewer(pcl[:,:3], col1, show_axis=False)
v.set(point_size=0.5, show_axis=False, show_info=True)